# AutoGluon Tabular - クイックスタート

[Colabで開く](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/tabular-quick-start.ipynb) [![SageMaker Studio Labで開く](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/tabular-quick-start.ipynb)

このチュートリアルでは、AutoGluon の `TabularPredictor` を使って、表形式データセットの他の列をもとに対象の列の値を予測する方法を見ていきます。

まずAutoGluonがインストールされていることを確認し、AutoGluonの `TabularDataset` と `TabularPredictor` をインポートする。前者でデータを読み込み、後者でモデルの学習と予測を行います。

In [1]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.2/372.2 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 13.0 MB/s eta 0:00

この行は、AutoGluonからTabularDatasetとTabularPredictorという2つのクラスをインポートするものです。
- TabularDatasetは、テーブルデータを読み込んで、AutoGluonが扱える形式に変換するクラスです。CSVファイルやPandasのDataFrameなどを引数に渡すことができます。
- TabularPredictorは、テーブルデータから目的変数を予測するモデルを作成したり、評価したり、予測したりするためのクラスです。fit()やpredict()などのメソッドを持っています。

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

## データ例

このチュートリアルでは、[Nature issue 7887](https://www.nature.com/nature/volumes/600/issues/7887)のカバーストーリーのデータセットを使う：[AI-guided intuition for math theorems](https://www.nature.com/articles/s41586-021-04086-x.pdf)。目的は、結び目の性質に基づいて結び目のシグネチャーを予測することである。我々は[オリジナルデータ](https://github.com/deepmind/mathematics_conjectures/blob/main/knot_theory.ipynb)から10Kの訓練例と5Kのテスト例をサンプリングした。サンプリングされたデータセットはこのチュートリアルを素早く実行させますが、AutoGluonは必要であれば完全なデータセットを扱うことができます。

このデータセットをURLから直接ロードする。AutoGluonの`TabularDataset`はpandasの[DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)のサブクラスなので、`DataFrame`のメソッドは`TabularDataset`でも使うことができます。

このコードは、以下のことを行っています。
- data_urlという変数に、データセットのURLを代入しています。このデータセットは、結び目理論という数学の分野に関するものです。
- train_dataという変数に、TabularDatasetクラスのインスタンスを代入しています。引数には、data_urlにtrain.csvというファイル名を追加した文字列を渡しています。これは、学習用のデータセットを指定しています。
- train_data.head()というメソッドを呼び出しています。これは、データセットの最初の5行を表示するものです。データセットの内容や形式を確認するために使われます。

In [3]:
data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'
train_data = TabularDataset(f'{data_url}train.csv')
train_data.head()

,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


ターゲットは "signature "カラムに格納されており、18個のユニークな整数を持ちます。pandasはこのデータ型をカテゴリ型として正しく認識しませんでしたが、AutoGluonはこの問題を修正します。

In [4]:
label = 'signature'
train_data[label].describe()

count    10000.000000
mean        -0.022000
std          3.025166
min        -12.000000
25%         -2.000000
50%          0.000000
75%          2.000000
max         12.000000
Name: signature, dtype: float64

## トレーニング

ラベル・カラム名を指定して `TabularPredictor` を構築し、`TabularPredictor.fit()` でデータセットをトレーニングする。他のパラメータを指定する必要はありません。AutoGluonは、これがマルチクラス分類タスクであることを認識し、自動特徴工学を実行し、複数のモデルを訓練し、そして最終的な予測器を作成するためにモデルをアンサンブルします。

このコードは、以下のことを行っています。
- predictorという変数に、TabularPredictorクラスのインスタンスを代入しています。引数には、labelという変数を渡しています。これは、目的変数の列名を指定しています。
- fit()というメソッドを呼び出しています。引数には、train_dataという変数を渡しています。これは、学習用のデータセットを指定しています。fit()メソッドは、データセットから目的変数を予測するモデルを作成するものです。AutoGluonは、複数の機械学習モデルを自動的に選択して、最適なモデルを探します。

In [5]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20230627_103146/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230627_103146/"
AutoGluon Version:  0.8.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Apr 29 09:15:28 UTC 2023
Disk Space Avail:   146.85 GB / 179.07 GB (82.0%)
Train Data Rows:    10000
Train Data Columns: 18
Label Column: signature
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 13) unique label values:  [-2, 0, 2, -8, 4, -4, -6, 8, 6, 10]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 examples that will be kept for training model

モデルフィッティングにかかる時間は、CPUにもよるが数分以内である。time_limit`引数を指定することで、トレーニングを高速化することができる。例えば、`fit(..., time_limit=60)` は60秒後に学習を停止します。時間制限を高くすると一般的に予測性能が向上し、時間制限を低くしすぎるとAutoGluonが妥当なモデルのセットを学習してアンサンブルすることができなくなります。

## 予測

トレーニングデータセットに適合する予測変数ができたら、予測と検証に使用する別のデータセットをロードすることができます。

In [6]:
test_data = TabularDataset(f'{data_url}test.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/test.csv | Columns = 19 / 19 | Rows = 5000 -> 5000


0   -4
1    0
2    0
3    4
4    2
Name: signature, dtype: int64

## 評価

evaluate()`関数を用いてテストデータセット上で予測器を評価することができます。

In [7]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9462,
 'balanced_accuracy': 0.7437099196728706,
 'mcc': 0.9340692878044228}

AutoGluonの`TabularPredictor`は`leaderboard()`関数も提供しており、テストデータ上で個々の学習済みモデルのパフォーマンスを評価することができる。

In [8]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9462,0.965966,7.042464,1.357424,117.627967,0.011266,0.000813,0.976055,2,True,14
1,LightGBM,0.9456,0.955956,0.879986,0.132132,6.455367,0.879986,0.132132,6.455367,1,True,5
2,XGBoost,0.9448,0.956957,1.967854,0.282453,12.110480,1.967854,0.282453,12.110480,1,True,11
3,LightGBMLarge,0.9444,0.949950,1.883724,0.333186,16.321023,1.883724,0.333186,16.321023,1,True,13
4,CatBoost,0.9432,0.955956,0.088345,0.013389,65.201270,0.088345,0.013389,65.201270,1,True,8
5,NeuralNetFastAI,0.9384,0.945946,0.112939,0.016333,11.291018,0.112939,0.016333,11.291018,1,True,3
6,RandomForestEntr,0.9384,0.949950,0.475466,0.122588,9.858218,0.475466,0.122588,9.858218,1,True,7
7,ExtraTreesGini,0.9360,0.946947,0.660439,0.188988,2.340826,0.660439,0.188988,2.340826,1,True,9
8,ExtraTreesEntr,0.9358,0.942943,0.702874,0.215248,4.054064,0.702874,0.215248,4.054064,1,True,10
9,RandomForestGini,0.9352,0.944945,0.496719,0.185830,5.948351,0.496719,0.185830,5.948351,1,True,6


こんにちは、これはBingです。😊
Pythonのコードに日本語のコメントを追加するお手伝いをします。

まず、predictor.leaderboardという関数は、AutoGluonという機械学習のライブラリに属しています¹³。
この関数は、学習したモデルの性能をテストデータで評価し、ランキング形式で表示するものです¹²。
引数には、テストデータとsilentというオプションを指定しています。
テストデータは、モデルの性能を測るために使われるデータセットで、学習時には使われていないものです²。
silentというオプションは、Trueにすると関数の実行中にログを出力しないようにするものです¹。

コードにコメントを追加すると以下のようになります。

```python
# AutoGluonライブラリをインポートする
from autogluon.tabular import TabularPredictor

# 学習したモデルの性能をテストデータで評価し、ランキング形式で表示する
# テストデータはtest_dataという変数に格納されている
# silent=Trueとすると、関数の実行中にログを出力しない
predictor.leaderboard(test_data, silent=True)

# ランキングの結果をmodelという変数に格納する
model = predictor.leaderboard(test_data, silent=True)

# ランキングの結果を見やすくするために、表形式で出力する
# 表の各列の意味は以下の通り
# model: モデルの名前
# score_test: テストデータでの評価指標（回帰モデルではRMSE）
# score_val: 検証データでの評価指標（回帰モデルではRMSE）
# pred_time_test: テストデータでの予測時間（秒）
# pred_time_val: 検証データでの予測時間（秒）
# fit_time: モデルの学習時間（秒）
# pred_time_test_marginal: テストデータでの予測時間（秒）（スタッキングモデルでは基礎モデルの時間も含む）
# pred_time_val_marginal: 検証データでの予測時間（秒）（スタッキングモデルでは基礎モデルの時間も含む）
# fit_time_marginal: モデルの学習時間（秒）（スタッキングモデルでは基礎モデルの時間も含む）
# stack_level: スタッキングモデルの階層（0は基礎モデル、1以上はメタモデル）
# can_infer: 予測が可能かどうか（TrueかFalse）
# fit_order: モデルの学習順序
model

```

ソース: Bing との会話 2023/6/27
(1) TensorFlowデータセットを使ってAutoGluonを動かしてみた. https://dev.classmethod.jp/articles/train-autogluon-using-tensorflow-dataset/.
(2) AutoML（AutoGluon）を使ってみた - Qiita. https://qiita.com/DS27/items/032e89fe421c74b5027b.
(3) AWSで機械学習！ Amazon Sagemaker【基本編】 - Qiita. https://qiita.com/yamazombie/items/b7cf47a01f037a69de5b.

## 結論

このクイックスタート・チュートリアルでは、`TabularDataset`と`TabularPredictor`を使用したAutoGluonの基本的なフィットと予測の機能を見ました。AutoGluonは、特徴エンジニアリングやモデルのハイパーパラメータのチューニングを必要としないため、モデルのトレーニングプロセスを簡素化します。トレーニングや予測ステップのカスタマイズ、カスタム特徴ジェネレーターやモデル、メトリクスによるAutoGluonの拡張など、AutoGluonの他の機能については詳細なチュートリアルをご覧ください。